## Beautiful Demo

# Hello

In [1]:
from matplotlib import pyplot as plt
from read_data import training_validation
from processing import integral_image
from time import time
from random import choice

training, validation = training_validation()

sample_image = choice(training)

tp_ii = time()

ii = integral_image(sample_image)

print("time to preprocess image: {}".format(time() - tp_ii))


plt.imshow(sample_image)
plt.show()



reading csv: 2.869016170501709
formatting to numpy matrices: 4.0106658935546875
picking random data: 0.20046710968017578
time to preprocess image: 0.017065048217773438


## Testing Naive Bayes 

In [3]:
from naive_bayes import naive_bayes
from numpy import zeros, arange
labels = arange(len(training))
classifier = naive_bayes()

(n_of_imagest, widtht, heigtht) = training.shape
into_linear_featurest = training.reshape(n_of_imagest, widtht * heigtht)

print(into_linear_featurest)

classifier.train(into_linear_featurest, labels)

(n_of_imagesv, widthv, heightv) = validation.shape
into_linear_featuresv = validation.reshape(n_of_imagesv, widthv * heightv)

print(classifier.predict(into_linear_featuresv[0]))

[[ 238.  236.  237. ...,   70.   75.   90.]
 [ 219.  215.  204. ...,    1.    1.    1.]
 [ 238.  236.  237. ...,   70.   75.   90.]
 ..., 
 [ 207.  208.  209. ...,  152.  149.  151.]
 [ 137.  107.   85. ...,   13.   10.    9.]
 [ 109.  109.  125. ...,  117.  108.  103.]]


KeyboardInterrupt: 